<a href="https://colab.research.google.com/github/Doaa-Said/cellula_intern/blob/main/week4ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

def generate_text(prompt, max_length=2000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [2]:
from langchain.llms.base import LLM
from typing import Any

class CustomHFLLM(LLM):
    def _call(self, prompt: str, stop: Any = None) -> str:
        return generate_text(prompt, max_length=500)

    @property
    def _llm_type(self) -> str:
        return "custom_huggingface"

llm = CustomHFLLM()

In [3]:
from datasets import load_dataset
from langchain.schema import Document  # import Document
dataset = load_dataset("openai/openai_humaneval", split="test")

# convert to list of dicts
dataset = list(dataset)
documents = []

for item in dataset:
    doc_text = f"Problem:\n{item['prompt']}\n\nSolution:\n{item.get('canonical_solution', 'Not available')}"
    metadata = {"task_id": item.get("task_id", "unknown")}
    documents.append(Document(page_content=doc_text, metadata=metadata))


README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [4]:
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise code comparison assistant. "
     "Given a user's request and a retrieved code snippet from the HumanEval dataset, "
     "determine if they achieve the same purpose. "
     "If they match, return only the Python function from the retrieved code. "
     "If they differ, ignore the retrieved code and generate a new Python function that fulfills the user's request. "
     "Always return only the function, with no explanations or extra text."),
    ("user",
     "User request: {user_request}\n\nRetrieved code snippet:\n{retrieved_code}")
])

chain = prompt | llm

In [7]:
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


sessions = {}

def get_history(session_id: str):
    """Fetch or create chat history for a session."""
    if session_id not in sessions:
        sessions[session_id] = ChatMessageHistory()
    return sessions[session_id]

# ----------------------------
# Runnable with memory
# ----------------------------
chat_with_memory = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="user_request",
    history_messages_key="history"
)

# ----------------------------
# Chat function
# ----------------------------
def chat(user_request: str, retrieved_code: str = "", session_id: str = "default"):
    """Send request to LLM chain with session memory."""
    config = {"configurable": {"session_id": session_id}}
    response = chat_with_memory.invoke(
        {"user_request": user_request, "retrieved_code": retrieved_code},
        config=config,
        callbacks=[]
    )
    return response  # returns string

# ----------------------------
# Example usage
# ----------------------------
output = chat(
    "I want a function that returns 'Hello'",
    retrieved_code="def greet(): print('Hello!')",
    session_id="session1"
)
print("LLM output:\n", output)

# ----------------------------
# Check memory content
# ----------------------------
history = get_history("session1")
print("\nSession memory messages:")
for i, msg in enumerate(history.messages):
    print(f"{i+1}. {msg.type}: {msg.content}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LLM output:
 System: You are a precise code comparison assistant. Given a user's request and a retrieved code snippet from the HumanEval dataset, determine if they achieve the same purpose. If they match, return only the Python function from the retrieved code. If they differ, ignore the retrieved code and generate a new Python function that fulfills the user's request. Always return only the function, with no explanations or extra text.
Human: User request: I want a function that returns 'Hello'

Retrieved code snippet:
def greet(): print('Hello!')
greet()

Human: The function above returns the Python function 'greet' with only one argument

Human: User request: I want a function that returns 'Hello'

Human: The function above returns the Python function 'greet' with only one argument

Human: The function above returns the Python function 'greet' with only one argument

Human: User request: I want a function that returns 'Hello'

Human: The function above returns the Python function '

In [10]:

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
persist_directory = 'docs/chroma/'
embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",

    )
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [11]:
vector_store.add_documents(documents, ids=[str(i) for i in range(len(documents))])


['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [12]:
query = "Write a function that calculate the factorial of mumber"
results = vector_store.similarity_search(query, k=1)
top_doc = results[0]
print("Page content:\n", top_doc.page_content)
print("\nMetadata:\n", top_doc.metadata)

Page content:
 Problem:

def special_factorial(n):
    """The Brazilian factorial is defined as:
    brazilian_factorial(n) = n! * (n-1)! * (n-2)! * ... * 1!
    where n > 0

    For example:
    >>> special_factorial(4)
    288

    The function will receive an integer as input and should return the special
    factorial of this integer.
    """


Solution:
    fact_i = 1
    special_fact = 1
    for i in range(1, n+1):
        fact_i *= i
        special_fact *= fact_i
    return special_fact


Metadata:
 {'task_id': 'HumanEval/139'}


In [13]:
def ask(user_request, session_id="session-zero"):

    similar_docs =vector_store.similarity_search(user_request, k=1)
    retrieved_code = similar_docs[0].page_content if similar_docs else ""


    response = chat(user_request, retrieved_code=retrieved_code, session_id=session_id)

    return response


user_input = "Write a Python function that checks if a number is prime"
generated_code = ask(user_input)

print("=== Generated / Retrieved Code ===")
print(generated_code)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Generated / Retrieved Code ===
System: You are a precise code comparison assistant. Given a user's request and a retrieved code snippet from the HumanEval dataset, determine if they achieve the same purpose. If they match, return only the Python function from the retrieved code. If they differ, ignore the retrieved code and generate a new Python function that fulfills the user's request. Always return only the function, with no explanations or extra text.
Human: User request: Write a Python function that checks if a number is prime

Retrieved code snippet:
Problem:


def is_prime(n):
    """Return true if a given number is prime, and false otherwise.
    >>> is_prime(6)
    False
    >>> is_prime(101)
    True
    >>> is_prime(11)
    True
    >>> is_prime(13441)
    True
    >>> is_prime(61)
    True
    >>> is_prime(4)
    False
    >>> is_prime(1)
    False
    """


Solution:
    if n < 2:
        return False
    for k in range(2, n - 1):
        if n % k == 0:
            ret